In [2]:
import h3
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

In [21]:
columns = next(open('gbif_bulk.csv', 'rb')).decode('utf-8').strip().split('\t')
cols_of_interest = {
    col: columns.index(col)
    for col in ('genus', 'species', 'decimalLatitude', 'decimalLongitude')
}
cols_of_interest

{'genus': 8, 'species': 9, 'decimalLatitude': 21, 'decimalLongitude': 22}

In [3]:
species = pd.read_csv('species_list.csv')
species_set = set([
    (row['genus'], row['species'])
    for _, row in species.iterrows()
])

In [34]:
new_rows = []
for i, row in tqdm(enumerate(open('gbif_bulk.csv', 'rb'))):
    if i == 0:
        continue
    cols = row.decode('utf-8').strip().split('\t')
    new_row = {
        col: cols[index]
        for col, index in cols_of_interest.items()
    }
    new_row['genus'] = new_row['genus'].lower()
    new_row['species'] = new_row['species'].lower().split(' ')[-1].strip()
    if (new_row['genus'], new_row['species']) in species_set:
        new_rows.append(new_row)
df = pd.DataFrame(new_rows)

9133862it [00:32, 284825.57it/s]


In [46]:
df = df.rename({'decimalLatitude': 'lat', 'decimalLongitude': 'lon'}, axis=1)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)
df.to_csv('gbif_positions.csv', index=False)

In [5]:
df = pd.read_csv('gbif_positions.csv')

In [19]:
rows = []

file_name = 'h3_positions_limited.csv'
with open(file_name, 'w') as fh:
    fh.write('hex_id,genus,species,count\n')

for genus, species in tqdm(species_set):
    sub_hexes = defaultdict(int)
    sub_df = df[(df['genus'] == genus) & (df['species'] == species)]
    for _, row in sub_df.iterrows():
        for zoom in range(9):
            sub_hexes[h3.geo_to_h3(row['lat'], row['lon'], zoom)] += 1
    lines = '\n'.join([
        f'{hex_id},{genus},{species},{count}'
        for hex_id, count in sub_hexes.items()
    ]) + '\n'
    with open(file_name, 'a') as fh:
        fh.write(lines)

100%|██████████| 1650/1650 [14:01<00:00,  1.96it/s]
